# Train Falcon model using SageMaker Distributed Data Parallel Library (SMDDP) and PyTorch Fully Sharded Data Parallelism (FSDP)

In this tutorial, we will show how to train or fine-tune [Falcon-7B-instruct](https://huggingface.co/tiiuae/falcon-7b-instruct) on the [Alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca) dataset.  We will use 2 p4d.24xlarge instances, which come with 8x NVIDIA A100 40GB GPUs along with the PyTorch Fully Sharded Data Parallelism (FSDP) technique to efficiently train this large model on limited GPU memory.  

To acclerate training speed, we will also use the **SageMaker Distributed Data Parallel Library (SMDDP)** which speeds up GPU communication across P4d instances during sharded data parallel training.  

Within `scripts/train.py` is the entry point for the training script where we initialize the SMDDP library

First, we'll install some dependencies necessary for this example

In [ ]:
!pip install "transformers" "datasets[s3]" "sagemaker" "boto3" --upgrade --quiet

In [ ]:
!pip install -r scripts/requirements.txt

If you are going to use Sagemaker in a local environment, you need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.



In [ ]:
import sagemaker,boto3

from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']


sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sagemaker_session is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sagemaker_session.default_bucket()

sagemaker_session = sagemaker.Session()
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sagemaker_session.default_bucket()}")
print(f"sagemaker session region: {sagemaker_session.boto_region_name}")


## 2. Load and prepare the dataset

As the base dataset, we will use the [Alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca) dataset, but before training the model, we need to preprocess the data. We will create chunks of `2048` tokens ([model max length](https://huggingface.co/EleutherAI/gpt-neox-20b)) to avoid unnecessary padding and computing. 

The first step is to load our dataset from Hugging Face.

In [ ]:
model_id = "tiiuae/falcon-7b"
dataset_name = "tatsu-lab/alpaca"

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load Tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load dataset from huggingface.co
dataset = load_dataset(dataset_name)

# downsample dataset to 10k
dataset = dataset.shuffle(42)

#### Split dataset into Train and Valid.

In [ ]:
if "validation" not in dataset.keys():
    dataset["validation"] = load_dataset(
        dataset_name,
        split="train[:5%]"
    )

    dataset["train"] = load_dataset(
        dataset_name,
        split="train[5%:]"
    )

The [Alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca) dataset contains 4 fields instruction, input , output and text. The text field is obtained by combining the remaining 3 fields and we will use the text field.

The last step of the data preparation is to tokenize and chunk our dataset. We convert our inputs (text) to token IDs by tokenizing, which the model can understand. Additionally, we concatenate our dataset samples into chunks of `2048` to avoid unnecessary padding.

In [ ]:

from itertools import chain
from functools import partial

def group_texts(examples,block_size = 2048):
        # Concatenate all texts.
        concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
        if total_length >= block_size:
            total_length = (total_length // block_size) * block_size
        # Split by chunks of max_len.
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result

column_names = dataset["train"].column_names

lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"],return_token_type_ids=False), batched=True, remove_columns=list(column_names)
).map(
    partial(group_texts, block_size=2048),
    batched=True,
)

We will start by saving the tokenized data locally .

In [ ]:
#save data locally

training_input_path = f'processed/data/'
lm_dataset.save_to_disk(training_input_path)

print(f"Saved data to: {training_input_path}")

## 3. Train the Falcon model using FSDP and SMDDP on Amazon SageMaker

We will begin by uploading the tokenized data to S3 which will be uploaded to the training cluster during training.

After we process the datasets we are going to use the new [FileSystem integration](https://huggingface.co/docs/datasets/filesystems) to upload our dataset to S3. We are using the `sagemaker_session.default_bucket()`, adjust this if you want to store the dataset in a different S3 bucket. We will use the S3 path later in our training script.

In [ ]:
training_input_path = f's3://{sagemaker_session.default_bucket()}/processed/data/'
print(f"training dataset to: {training_input_path}")# save train_dataset to s3
lm_dataset.save_to_disk(training_input_path)

print(f"uploaded data to: {training_input_path}")

As mentioned in the beginning, we will use Amazon SageMaker and PyTorch FSDP to train our model. Amazon SageMaker makes it easy to create a multi-node cluster to train our model in a distributed manner. The `sagemaker` python SDK supports running training jobs using `torchrun`, to distribute the script across multiple nodes and GPUs. 

To use `torchrun` to execute our scripts, we only have to define the `distribution` parameter in our Estimator and set it to `"torch_distributed": {"enabled": True}`.

In our example, we will use `full shard auto_wrap` and `FalconDecoderLayer` as transformer layer policy. If you run this example and change the model, make sure to also adjust the transformer layer policy in `scripts/train.py`.

Within the entry point of the training script, we also import SMDDP and use it as the backend for the process group. This enables faster communication between P4d instances than the open source Nvidia Collective Communications Library (NCCL) and ultimately speeds up training.  

To create a sagemaker training job, we create a `PyTorch` Estimator and provide all our information. SageMaker takes care of starting and managing all the required EC2 instances for us, uploads the provided scripts and downloads the data from our S3 bucket into the container at `/opt/ml/input/data`. Then, it starts the training job by running.

Note that SageMaker by default uses the latest [AWS Deep Learning Container (DLC)](https://aws.amazon.com/machine-learning/containers/), so you can comment out the `ecr_image` variable if you don't want to use your own custom image built from a DLC. Also note that if using FSx when launching the SageMaker notebook instance, you will need to use the same `subnet` and `security_group_config`.  

In [ ]:
import time


ecr_image="<ECR_IMAGE_URI"
job_name = f'huggingface-fsdp-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
subnet_config = ["<SUBNET_ID>"]
security_group_config = ["<SECURITY_GROUP_ID>"]

# hyperparameters, which are passed into the training job
hyperparameters={
    'model_id': model_id, # model id from huggingface.co/models
    'dataset_path': '/opt/ml/input/data/train', # path where sagemaker will save training dataset
    'valid_path':"/opt/ml/input/data/valid",
    'gradient_checkpointing': True, # enable gradient checkpointing
    'bf16': True, # enable mixed precision training
    'optimizer': "adamw_torch", # optimizer
    'per_device_train_batch_size': 1, # batch size per device during training
    'epochs': 1, # number of epochs to train
    'fsdp': '"full_shard auto_wrap"', # fully sharded data parallelism
    'cache_dir': "/opt/ml/sagemaker/warmpoolcache", #change this to /tmp if not using warmpools
    'max_steps':30
}

# estimator 
estimator = PyTorch(
  entry_point="train.py",
  max_run=1800,
  job_name=job_name,
  role=role,
  framework_version="2.0.1",
  py_version="py310",
  image_uri=ecr_image,
  source_dir="./scripts",
  instance_count=2,
  instance_type="ml.p4d.24xlarge",
  sagemaker_session=sagemaker_session,
  subnets=subnet_config,
  security_group_ids=security_group_config,
  disable_output_compression=True,
  distribution={"torch_distributed": {"enabled": True}},
  keep_alive_period_in_seconds=600,
  hyperparameters=hyperparameters,
)


We can now start our training job, with the `.fit()` method passing our S3 path to the training script.

In [ ]:
# define a data input dictonary with our uploaded s3 uris
data = {'train': training_input_path}

# starting the train job with our uploaded datasets as input
estimator.fit(data, wait=True)

### Terminate the warm pool cluster if no longer needed

In [ ]:
sess.update_training_job(estimator.latest_training_job.job_name, resource_config={"KeepAlivePeriodInSeconds":0})